# 模型壓縮 (Model Compression)

## 學習目標
- 理解模型壓縮的必要性
- 實作知識蒸餾 (Knowledge Distillation)
- 學習網路剪枝 (Pruning)
- 掌握模型量化 (Quantization)

## 對應課程
- [李宏毅 ML 2021 - Network Compression](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW13: Network Compression](https://github.com/ga642381/ML2021-Spring/tree/main/HW13)

## 模型壓縮概述

```
為什麼需要壓縮？
├── 邊緣設備資源有限
├── 降低推理延遲
├── 減少能耗
└── 降低部署成本

主要方法:
├── Knowledge Distillation（知識蒸餾）
│   用小模型學習大模型的知識
├── Pruning（剪枝）
│   移除不重要的權重或結構
├── Quantization（量化）
│   降低數值精度
└── Architecture Design
    設計高效的網路結構
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
torch.manual_seed(42)
np.random.seed(42)

---
## Part 1: 準備工作 - Teacher 和 Student 模型

In [ ]:
# ========== 定義模型 ==========

class TeacherModel(nn.Module):
    """
    大型 Teacher 模型
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class StudentModel(nn.Module):
    """
    小型 Student 模型
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 比較模型大小
teacher = TeacherModel()
student = StudentModel()

teacher_params = sum(p.numel() for p in teacher.parameters())
student_params = sum(p.numel() for p in student.parameters())

print(f"Teacher 模型參數量: {teacher_params:,}")
print(f"Student 模型參數量: {student_params:,}")
print(f"壓縮比: {teacher_params / student_params:.1f}x")

In [ ]:
# ========== 準備資料 ==========

# 使用 CIFAR-10 子集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 模擬資料（避免下載）
class FakeDataset(torch.utils.data.Dataset):
    def __init__(self, size=1000):
        self.size = size
        self.data = torch.randn(size, 3, 32, 32)
        self.targets = torch.randint(0, 10, (size,))
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

train_dataset = FakeDataset(2000)
test_dataset = FakeDataset(500)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"訓練集: {len(train_dataset)} 樣本")
print(f"測試集: {len(test_dataset)} 樣本")

---
## Part 2: Knowledge Distillation（知識蒸餾）

讓 Student 模型學習 Teacher 模型的「軟標籤」（soft labels）。

### 損失函數

$$L = \alpha \cdot L_{CE}(y, p_s) + (1-\alpha) \cdot T^2 \cdot L_{KL}(p_t^{(T)}, p_s^{(T)})$$

- $L_{CE}$: 與真實標籤的交叉熵
- $L_{KL}$: 與 Teacher 軟標籤的 KL 散度
- $T$: 溫度參數
- $p^{(T)} = \text{softmax}(z/T)$

In [ ]:
# ========== 知識蒸餾損失 ==========

class DistillationLoss(nn.Module):
    """
    知識蒸餾損失函數
    """
    
    def __init__(self, temperature=4.0, alpha=0.7):
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')
    
    def forward(self, student_logits, teacher_logits, labels):
        # 硬標籤損失
        hard_loss = self.ce_loss(student_logits, labels)
        
        # 軟標籤損失
        soft_student = F.log_softmax(student_logits / self.temperature, dim=1)
        soft_teacher = F.softmax(teacher_logits / self.temperature, dim=1)
        soft_loss = self.kl_loss(soft_student, soft_teacher) * (self.temperature ** 2)
        
        # 總損失
        total_loss = self.alpha * hard_loss + (1 - self.alpha) * soft_loss
        
        return total_loss, hard_loss.item(), soft_loss.item()

# 測試
distill_loss = DistillationLoss(temperature=4.0, alpha=0.7)
student_out = torch.randn(4, 10)
teacher_out = torch.randn(4, 10)
labels = torch.randint(0, 10, (4,))

loss, hard, soft = distill_loss(student_out, teacher_out, labels)
print(f"蒸餾損失: {loss.item():.4f}")
print(f"  硬標籤損失: {hard:.4f}")
print(f"  軟標籤損失: {soft:.4f}")

In [ ]:
# ========== 訓練 Teacher ==========

def train_model(model, train_loader, test_loader, epochs=10, lr=1e-3):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    history = {'train_loss': [], 'test_acc': []}
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # 測試
        model.eval()
        correct = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                correct += (out.argmax(1) == y).sum().item()
        
        acc = correct / len(test_loader.dataset)
        history['train_loss'].append(total_loss / len(train_loader))
        history['test_acc'].append(acc)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f}, Acc={acc:.4f}")
    
    return model, history

# 訓練 Teacher
print("訓練 Teacher 模型...")
teacher, teacher_history = train_model(teacher, train_loader, test_loader, epochs=10)

In [ ]:
# ========== 知識蒸餾訓練 Student ==========

def train_with_distillation(student, teacher, train_loader, test_loader, epochs=10, lr=1e-3):
    student = student.to(device)
    teacher = teacher.to(device)
    teacher.eval()  # Teacher 固定
    
    optimizer = torch.optim.Adam(student.parameters(), lr=lr)
    criterion = DistillationLoss(temperature=4.0, alpha=0.3)
    
    history = {'train_loss': [], 'test_acc': []}
    
    for epoch in range(epochs):
        student.train()
        total_loss = 0
        
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            
            # Teacher 輸出（不需要梯度）
            with torch.no_grad():
                teacher_out = teacher(x)
            
            # Student 輸出
            optimizer.zero_grad()
            student_out = student(x)
            
            # 蒸餾損失
            loss, _, _ = criterion(student_out, teacher_out, y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # 測試
        student.eval()
        correct = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                out = student(x)
                correct += (out.argmax(1) == y).sum().item()
        
        acc = correct / len(test_loader.dataset)
        history['train_loss'].append(total_loss / len(train_loader))
        history['test_acc'].append(acc)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f}, Acc={acc:.4f}")
    
    return student, history

# 知識蒸餾訓練
print("\n使用知識蒸餾訓練 Student...")
student_distilled = StudentModel()
student_distilled, distill_history = train_with_distillation(
    student_distilled, teacher, train_loader, test_loader, epochs=10
)

# 對比：直接訓練 Student
print("\n直接訓練 Student...")
student_direct = StudentModel()
student_direct, direct_history = train_model(student_direct, train_loader, test_loader, epochs=10)

In [ ]:
# ========== 比較結果 ==========

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 準確率比較
axes[0].plot(teacher_history['test_acc'], label='Teacher', marker='o')
axes[0].plot(distill_history['test_acc'], label='Student (Distilled)', marker='s')
axes[0].plot(direct_history['test_acc'], label='Student (Direct)', marker='^')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Test Accuracy')
axes[0].set_title('Accuracy Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 模型大小和準確率
models = ['Teacher', 'Student\n(Distilled)', 'Student\n(Direct)']
params = [teacher_params, student_params, student_params]
accs = [teacher_history['test_acc'][-1], distill_history['test_acc'][-1], direct_history['test_acc'][-1]]

x = np.arange(len(models))
width = 0.35

ax2 = axes[1]
bars1 = ax2.bar(x - width/2, [p/1000 for p in params], width, label='Parameters (K)')
ax2.set_ylabel('Parameters (K)')
ax2.set_xlabel('Model')
ax2.set_xticks(x)
ax2.set_xticklabels(models)

ax3 = ax2.twinx()
bars2 = ax3.bar(x + width/2, [a*100 for a in accs], width, color='orange', label='Accuracy (%)')
ax3.set_ylabel('Accuracy (%)')

ax2.legend(loc='upper left')
ax3.legend(loc='upper right')
ax2.set_title('Model Size vs Accuracy')

plt.tight_layout()
plt.show()

print(f"\n結果總結:")
print(f"  Teacher 準確率: {teacher_history['test_acc'][-1]:.4f}")
print(f"  Distilled Student: {distill_history['test_acc'][-1]:.4f}")
print(f"  Direct Student: {direct_history['test_acc'][-1]:.4f}")

---
## Part 3: Pruning（剪枝）

移除模型中不重要的權重或結構。

In [ ]:
# ========== 非結構化剪枝 ==========

def apply_unstructured_pruning(model, amount=0.3):
    """
    應用全局非結構化剪枝
    
    移除最小的 amount 比例的權重
    """
    parameters_to_prune = []
    
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            parameters_to_prune.append((module, 'weight'))
    
    # 全局剪枝
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=amount,
    )
    
    return model

def get_sparsity(model):
    """
    計算模型稀疏度
    """
    total = 0
    zeros = 0
    
    for name, param in model.named_parameters():
        if 'weight' in name:
            total += param.numel()
            zeros += (param == 0).sum().item()
    
    return zeros / total

# 測試剪枝
pruned_model = copy.deepcopy(student_distilled)

print(f"剪枝前稀疏度: {get_sparsity(pruned_model):.2%}")

apply_unstructured_pruning(pruned_model, amount=0.5)

print(f"剪枝後稀疏度: {get_sparsity(pruned_model):.2%}")

In [ ]:
# ========== 評估剪枝效果 ==========

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    
    start_time = time.time()
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            correct += (out.argmax(1) == y).sum().item()
    
    inference_time = time.time() - start_time
    accuracy = correct / len(test_loader.dataset)
    
    return accuracy, inference_time

# 不同剪枝率的效果
sparsity_levels = [0.0, 0.3, 0.5, 0.7, 0.9]
results = []

for sparsity in sparsity_levels:
    model = copy.deepcopy(student_distilled)
    model = model.to(device)
    
    if sparsity > 0:
        apply_unstructured_pruning(model, amount=sparsity)
    
    acc, time_taken = evaluate_model(model, test_loader)
    actual_sparsity = get_sparsity(model)
    
    results.append({
        'target_sparsity': sparsity,
        'actual_sparsity': actual_sparsity,
        'accuracy': acc,
        'time': time_taken
    })
    
    print(f"稀疏度 {sparsity*100:.0f}%: Acc={acc:.4f}, Time={time_taken:.4f}s")

# 可視化
fig, ax = plt.subplots(figsize=(10, 6))

sparsities = [r['actual_sparsity']*100 for r in results]
accuracies = [r['accuracy']*100 for r in results]

ax.plot(sparsities, accuracies, 'bo-', markersize=10)
ax.set_xlabel('Sparsity (%)')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Accuracy vs Sparsity')
ax.grid(True, alpha=0.3)

for i, (s, a) in enumerate(zip(sparsities, accuracies)):
    ax.annotate(f'{a:.1f}%', (s, a), textcoords="offset points", xytext=(0,10))

plt.tight_layout()
plt.show()

---
## Part 4: Quantization（量化）

將模型從 FP32 轉換為更低精度（如 INT8）。

In [ ]:
# ========== 動態量化 ==========

def dynamic_quantize(model):
    """
    動態量化（主要針對線性層）
    """
    quantized_model = torch.quantization.quantize_dynamic(
        model,
        {nn.Linear},
        dtype=torch.qint8
    )
    return quantized_model

def get_model_size(model):
    """
    獲取模型大小（MB）
    """
    torch.save(model.state_dict(), 'temp_model.pt')
    import os
    size_mb = os.path.getsize('temp_model.pt') / 1e6
    os.remove('temp_model.pt')
    return size_mb

# 量化模型
original_model = copy.deepcopy(student_distilled).cpu()
quantized_model = dynamic_quantize(original_model)

print(f"原始模型大小: {get_model_size(original_model):.2f} MB")
print(f"量化模型大小: {get_model_size(quantized_model):.2f} MB")

In [ ]:
# ========== 模擬 INT8 量化 ==========

class QuantizedLinear(nn.Module):
    """
    模擬的 INT8 量化線性層
    """
    
    def __init__(self, linear_layer):
        super().__init__()
        
        # 量化權重
        weight = linear_layer.weight.data
        self.scale = weight.abs().max() / 127
        self.weight_int8 = torch.round(weight / self.scale).clamp(-128, 127).to(torch.int8)
        
        # 偏置保持 FP32
        self.bias = linear_layer.bias
    
    def forward(self, x):
        # 反量化權重
        weight_fp32 = self.weight_int8.float() * self.scale
        return F.linear(x, weight_fp32, self.bias)

def quantize_model_manual(model):
    """
    手動量化模型中的線性層
    """
    for name, module in model.named_children():
        if isinstance(module, nn.Linear):
            setattr(model, name, QuantizedLinear(module))
        elif len(list(module.children())) > 0:
            quantize_model_manual(module)
    return model

print("手動量化示範完成")

---
## 🎯 總結

### 壓縮方法比較

| 方法 | 壓縮率 | 精度損失 | 硬體要求 |
|------|--------|----------|----------|
| **Knowledge Distillation** | 2-10x | 小 | 無特殊要求 |
| **Pruning** | 2-10x | 中等 | 稀疏運算支持 |
| **Quantization** | 2-4x | 小 | INT8 支持 |

### 李宏毅 HW13 技巧

```
Knowledge Distillation:
1. 選擇適當的溫度 T（通常 3-10）
2. 調整 alpha 平衡硬/軟標籤
3. 可以組合多個 Teacher

Pruning:
1. 逐步剪枝比一次性剪枝效果好
2. 剪枝後需要 fine-tune
3. 結構化剪枝更容易加速

Quantization:
1. PTQ vs QAT 權衡
2. 混合精度量化
3. 敏感層保持高精度
```

### 下一步

前往 `advanced_learning/meta_learning.ipynb` 學習元學習！

---
## 🏋️ 練習

### 練習 1: 比較不同蒸餾溫度的效果

測試溫度參數 T 對知識蒸餾效果的影響。

In [ ]:
# ========== 練習 1: 比較不同蒸餾溫度 ==========

def train_with_temperature(student, teacher, train_loader, test_loader, temperature, epochs=10):
    """
    使用指定溫度進行知識蒸餾
    """
    student = student.to(device)
    teacher = teacher.to(device)
    teacher.eval()
    
    optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)
    criterion = DistillationLoss(temperature=temperature, alpha=0.3)
    
    for epoch in range(epochs):
        student.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                teacher_out = teacher(x)
            optimizer.zero_grad()
            student_out = student(x)
            loss, _, _ = criterion(student_out, teacher_out, y)
            loss.backward()
            optimizer.step()
    
    # 評估
    student.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = student(x)
            correct += (out.argmax(1) == y).sum().item()
    
    return correct / len(test_loader.dataset)

def experiment_temperature():
    """
    比較不同溫度參數的效果
    """
    temperatures = [1, 2, 4, 8, 16]
    results = []
    
    print("練習 1: 蒸餾溫度實驗")
    print("=" * 50)
    
    for T in temperatures:
        student = StudentModel()
        acc = train_with_temperature(student, teacher, train_loader, test_loader, temperature=T, epochs=10)
        results.append({'temperature': T, 'accuracy': acc})
        print(f"溫度 T={T:2d}: 準確率 = {acc:.4f}")
    
    return results

# 執行實驗
temp_results = experiment_temperature()

# 視覺化
fig, ax = plt.subplots(figsize=(10, 6))

temps = [r['temperature'] for r in temp_results]
accs = [r['accuracy'] * 100 for r in temp_results]

bars = ax.bar(range(len(temps)), accs, color='steelblue')
ax.set_xticks(range(len(temps)))
ax.set_xticklabels([f'T={t}' for t in temps])
ax.set_xlabel('Temperature')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Effect of Distillation Temperature')

for bar, acc in zip(bars, accs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            f'{acc:.1f}%', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n結論：")
print("- T=1 等於普通的 softmax，軟標籤不夠「軟」")
print("- T 過大會使分佈過於平滑，丟失類別資訊")
print("- 通常 T=3~10 是較好的選擇")